# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [2]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1452487538 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/pedrohserrano/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1452487538 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to ph.serrano@hotmail.com and will expire on September 24, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-565 - Server binary: /Users/pedrohserrano/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452487538.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 2.135636     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 2.309358     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [7]:
weight_summary_all = model_all.get("coefficients")
none_zero_all = model_all['coefficients']['value'].nnz()
none_zero_all

6

In [15]:
weight_summary_all.print_rows(18,3)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

(intercept), bathrooms, sqft_living, sqft_living_sqrt MAL
hay que sacar todos los no ceros
grade, sqft_living

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [14]:
#l1_range = np.logspace(8, 10, num=20)
l1_range = np.logspace(1, 7, num=13)
i = 0
for l1_pa in l1_range:
    model2 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pa,verbose = False)
    none_zero = model2['coefficients']['value'].nnz()
    i = i+1
    print 'i = ',i, 'L1P = ', l1_pa, 'None Zero Value = ',none_zero 

i =  1 L1P =  10.0 None Zero Value =  18
i =  2 L1P =  31.6227766017 None Zero Value =  18
i =  3 L1P =  100.0 None Zero Value =  18
i =  4 L1P =  316.227766017 None Zero Value =  18
i =  5 L1P =  1000.0 None Zero Value =  18
i =  6 L1P =  3162.27766017 None Zero Value =  18
i =  7 L1P =  10000.0 None Zero Value =  18
i =  8 L1P =  31622.7766017 None Zero Value =  18
i =  9 L1P =  100000.0 None Zero Value =  18
i =  10 L1P =  316227.766017 None Zero Value =  18
i =  11 L1P =  1000000.0 None Zero Value =  18
i =  12 L1P =  3162277.66017 None Zero Value =  18
i =  13 L1P =  10000000.0 None Zero Value =  18


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [17]:
L1_penalty = np.logspace(1, 7, num=13)
RSS = np.zeros(len(L1_penalty))
for i in range(len(L1_penalty)):
    lp = L1_penalty[i]
    m = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, verbose = False,l2_penalty=0., l1_penalty=lp)
    p = m.predict(validation)
    res = p-validation['price']
    RSS[i] = (res*res).sum()
    print 'i = ',i, 'L1P = ', lp, 'RSS = ',RSS[i]

 i =  0 L1P =  10.0 RSS =  6.25766285142e+14
i =  1 L1P =  31.6227766017 RSS =  6.25766285362e+14
i =  2 L1P =  100.0 RSS =  6.25766286058e+14
i =  3 L1P =  316.227766017 RSS =  6.25766288257e+14
i =  4 L1P =  1000.0 RSS =  6.25766295212e+14
i =  5 L1P =  3162.27766017 RSS =  6.25766317206e+14
i =  6 L1P =  10000.0 RSS =  6.25766386761e+14
i =  7 L1P =  31622.7766017 RSS =  6.25766606749e+14
i =  8 L1P =  100000.0 RSS =  6.25767302792e+14
i =  9 L1P =  316227.766017 RSS =  6.25769507644e+14
i =  10 L1P =  1000000.0 RSS =  6.25776517727e+14
i =  11 L1P =  3162277.66017 RSS =  6.25799062845e+14
i =  12 L1P =  10000000.0 RSS =  6.25883719085e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

Q2: between 0 and 100 When l1_penalty = 10 validation set has lowest rss.

Q3: 18 is correct.

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [20]:
l1_range = np.logspace(8, 10, num=20)
#l1_range = np.logspace(1, 7, num=13)
i = 0
for l1_pa in l1_range:
    model2 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pa,verbose = False)
    none_zero = model2['coefficients']['value'].nnz()
    i = i+1
    print 'i = ',i, 'L1P = ', l1_pa, 'None Zero Value = ',none_zero 

i =  1 L1P =  100000000.0 None Zero Value =  18
i =  2 L1P =  127427498.57 None Zero Value =  18
i =  3 L1P =  162377673.919 None Zero Value =  18
i =  4 L1P =  206913808.111 None Zero Value =  18
i =  5 L1P =  263665089.873 None Zero Value =  17
i =  6 L1P =  335981828.628 None Zero Value =  17
i =  7 L1P =  428133239.872 None Zero Value =  17
i =  8 L1P =  545559478.117 None Zero Value =  17
i =  9 L1P =  695192796.178 None Zero Value =  17
i =  10 L1P =  885866790.41 None Zero Value =  16
i =  11 L1P =  1128837891.68 None Zero Value =  15
i =  12 L1P =  1438449888.29 None Zero Value =  15
i =  13 L1P =  1832980710.83 None Zero Value =  13
i =  14 L1P =  2335721469.09 None Zero Value =  12
i =  15 L1P =  2976351441.63 None Zero Value =  10
i =  16 L1P =  3792690190.73 None Zero Value =  6
i =  17 L1P =  4832930238.57 None Zero Value =  5
i =  18 L1P =  6158482110.66 None Zero Value =  3
i =  19 L1P =  7847599703.51 None Zero Value =  1
i =  20 L1P =  10000000000.0 None Zero Value =  

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

the min is where i = 15

In [ ]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [21]:
#Q4 is the i = 16.   3792690190  bien

In [ ]:
Q5 2976351441  mal
3592000000 mal
206913808

In [ ]:
sqft_living
sqft_lot_sqrt
bathrooms

mal

sqft_living
sqft_lot_sqrt

mal

sqft_living
bedrooms_square

mal

sqft_living
bedrooms_square
sqft_lot_sqrt

mal

sqft_living

mal

bedrooms_square
sqft_lot_sqrt